In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import numpy as np
traval_tra_features = np.load('traval_tra_features.npy').astype(np.float32)
traval_tra_targets = np.load('traval_tra_targets.npy').astype(np.int64)[:,0]
traval_val_features = np.load('traval_val_features.npy').astype(np.float32)
traval_val_targets = np.load('traval_val_targets.npy').astype(np.int64)[:,0]
traval_tra_features.shape, traval_val_features.shape

((687636, 2600), (76405, 2600))

In [2]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
import sklearn
from skorch.callbacks import EpochScoring, LRScheduler, Checkpoint
from torch.optim import Adam, SGD
import adamod

/home/amax/miniconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
from skorch.dataset import Dataset
from skorch.helper import predefined_split
valid_ds = Dataset(traval_val_features, traval_val_targets)

In [4]:
class DownSample2x(nn.Sequential):
    def __init__(self, _in, _out):
        super().__init__(
            nn.Conv1d(_in, _out, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
        )

class SELayer(nn.Module):
    def __init__(self, _in, _hidden=64):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(_in, _hidden),
            nn.PReLU(),
            nn.Linear(_hidden, _in),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y
    
class ResConv1d(nn.Module):
    def __init__(self, _in, _out):
        super(ResConv1d, self).__init__()
        
        self.cal = nn.Sequential(
            nn.Conv1d(_in, _out, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm1d(_out),
            nn.ReLU(),
            nn.Conv1d(_out, _out, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm1d(_out),
        )
        self.se = SELayer(_out, _out)
        self.conv = nn.Conv1d(_in, _out, kernel_size=1, padding=0, stride=1)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(_out)
        
    def forward(self, x):
        res = self.cal(x)
        res = self.se(res)
        
        x = self.bn(self.conv(x))
        
        return self.relu(res + x)
        
class MyModule(nn.Module):
    def __init__(self, nonlin=F.relu):
        super(MyModule, self).__init__()
        
        self.d1 = DownSample2x(1, 64)
        self.c1 = ResConv1d(64, 64)
        
        self.d2 = DownSample2x(64, 64)
        self.c2 = ResConv1d(64, 64)
        
        self.d3 = DownSample2x(64, 64)
        self.c3 = ResConv1d(64, 64)
        
        self.d4 = DownSample2x(64, 64)
        self.c4 = ResConv1d(64, 64)
        
        self.d5 = DownSample2x(64, 64)
        self.c5 = ResConv1d(64, 64)
        
        self.d6 = DownSample2x(64, 64)
        self.c6 = ResConv1d(64, 64)
        
        self.dropout = nn.Dropout(0.5)
        self.cls = nn.Linear(64 * 40, 3)
        
    def preprocess(self, x, p=2, eps=1e-8):
        x = x / (x.norm(p=p, dim=1, keepdim=True)+eps)
        x = x.unsqueeze(1)
        return x
        
    def forward(self, x):
        bs = x.size(0)
        x = self.preprocess(x)
        
        x = self.d1(x)
        x = self.c1(x)
        
        x = self.d2(x)
        x = self.c2(x)
        
        x = self.d3(x)
        x = self.c3(x)

        x = self.d4(x)
        x = self.c4(x)
        
        x = self.d5(x)
        x = self.c5(x)
        
        x = self.d6(x)
        x = self.c6(x)
        
        x = x.reshape(bs, -1)
        x = self.dropout(x)

        return F.softmax(self.cls(x))

    
def microf1(net, ds, y=None):
    y_true = [y for _, y in ds]
    y_pred = net.predict(ds)
    return sklearn.metrics.f1_score(y_true, y_pred,average='micro')
def macrof1(net, ds, y=None):
    y_true = [y for _, y in ds]
    y_pred = net.predict(ds)
    return sklearn.metrics.f1_score(y_true, y_pred,average='macro')

class F1Loss(nn.Module):
    def __init__(self):
        super(F1Loss, self).__init__()

    def forward(self, input, target, eps=1e-10):
        loss = 0
        for idx, i in enumerate(torch.eye(3).cuda()):
            t = i.view(3,1)
            y_pred_ = input.matmul(t).squeeze()
            y_true_ = target==idx
            loss += 0.5 * (y_true_ * y_pred_).sum() / (y_true_ + y_pred_ + eps).sum()
        return -torch.log(loss+eps)

## General Training

In [5]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model31')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9673     0.9919        0.0338       0.9919        0.0258     +  307.5992
      2     0.9139     0.9763        0.0203       0.9763        0.0726        310.0884
      3     0.9788     0.9945        0.0177       0.9945        0.0184     +  306.0065
      4     0.9789     0.9946        0.0161       0.9946        0.0186     +  304.8534
      5     0.9789     0.9948        0.0150       0.9948        0.0171        305.3079
      6     0.9826     0.9957        0.0144       0.9957        0.0144     +  304.0510
      7     0.9809     0.9951        0.0137       0.9951        0.0155        303.9848
      8     0.9829     0.9957        0.0131       0.9957        0.0141     +  303.1474
      9     0.9812     0.9949        0.0125       0.9949        0.0174        302.1176
     10     0.9779     0.9947        0.0119

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

## MacroLoss Fintuning

In [7]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model31', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model31/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9837     0.9960        0.3042       0.9960        0.3096     +  312.0657
      2     0.9840     0.9961        0.3023       0.9961        0.3095     +  312.1629
      3     0.9845     0.9961        0.3023       0.9961        0.3090     +  311.5738
      4     0.9846     0.9962        0.3016       0.9962        0.3088     +  313.2576
      5     0.9844     0.9961        0.3016       0.9961        0.3090        312.6374
      6     0.9842     0.9961        0.3002       0.9961        0.3095        311.8045
      7     0.9836     0.9960        0.3003       0.9960        0.3100        312.5994
      8     0.9836     0.9959        0.3004       0.9959        0.3096        310.8952
      9     0.9843     0.9961        0.3009       0.9961        0.3093        312.1927
     10     0.9836     0.9960        0.2996

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [8]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model32')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model32', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model32/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9672     0.9917        0.0344       0.9917        0.0269     +  305.1364
      2     0.9807     0.9950        0.0203       0.9950        0.0172     +  306.0134
      3     0.9799     0.9949        0.0177       0.9949        0.0168        306.2343
      4     0.9784     0.9949        0.0161       0.9949        0.0158        306.6969
      5     0.9801     0.9952        0.0150       0.9952        0.0152        305.8357
      6     0.9804     0.9951        0.0144       0.9951        0.0154        305.5180
      7     0.9811     0.9953        0.0137       0.9953        0.0158     +  304.8416
      8     0.9832     0.9955        0.0129       0.9955        0.0156     +  305.2161
      9     0.9807     0.9954        0.0126       0.9954        0.0153        306.0027
     10     0.9838     0.9959        0.0120

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [9]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model33')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model33', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model33/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9026     0.9767        0.0341       0.9767        0.0699     +  302.4418
      2     0.9780     0.9944        0.0206       0.9944        0.0179     +  303.2581
      3     0.9775     0.9943        0.0175       0.9943        0.0194        301.4525
      4     0.9811     0.9954        0.0162       0.9954        0.0151     +  302.0274
      5     0.9799     0.9949        0.0152       0.9949        0.0159        301.9294
      6     0.9807     0.9951        0.0143       0.9951        0.0155        301.2386
      7     0.9821     0.9956        0.0137       0.9956        0.0152     +  302.8372
      8     0.9817     0.9954        0.0132       0.9954        0.0154        300.0040
      9     0.9796     0.9952        0.0126       0.9952        0.0163        302.2090
     10     0.9812     0.9955        0.0121

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [10]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model34')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model34', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model34/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9726     0.9930        0.0346       0.9930        0.0232     +  305.8387
      2     0.9739     0.9933        0.0204       0.9933        0.0221     +  302.1640
      3     0.9776     0.9946        0.0178       0.9946        0.0194     +  302.9010
      4     0.9765     0.9946        0.0161       0.9946        0.0182        303.9640
      5     0.9825     0.9958        0.0149       0.9958        0.0139     +  301.6424
      6     0.9826     0.9956        0.0142       0.9956        0.0144     +  301.1953
      7     0.9812     0.9955        0.0136       0.9955        0.0148        303.1717
      8     0.9811     0.9953        0.0130       0.9953        0.0150        303.0298
      9     0.9833     0.9959        0.0123       0.9959        0.0136     +  302.7733
     10     0.9824     0.9957        0.0120

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 